<a href="https://colab.research.google.com/github/mateoLoxzDigital/poc_repo/blob/main/POC_Function_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# build basic Bee Plugin Template Flask App and the Marlowe button just submits the data to the Loxz Digital API

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/MailUp_Project/data/wrangled_data.csv')

## Regression Prediction Just  On Selected Variable

(consideration, do we want to filter out other industries from training dataset?) (the industry weights are taken into account but filtering them entirely might change probabilities) (I guess just test it out???)

In [4]:
data.head(2)

,Unnamed: 0,label,text,length,title,title_length,click_through,abandoned_cart,unsubscribed,open_rate,num_pics,num_videos,video_length,length_binned,title_length_binned,campain_type,industry
0,0,0,"Go until jurong point, crazy.. Available only ...",20,"Go until jurong point, crazy",5,0,1,1,1,2,2,69,med,med,Promotional,Medical
1,1,0,Ok lar... Joking wif u oni...,6,Ok lar,2,1,0,1,0,1,3,107,small,small,Survey,Hospitality


In [12]:
target_variables = ['label', 'unsubscribed', 'open_rate', 'click_through', 'abandoned_cart']
campaign_types = ['Promotional', 'Survey', 'No_Opener', 'Revenue_Based', 'abandoned_cart', 'engagement_campaign']
industry_types = ['Medical', 'Hospitality', 'Industrial', 'Automotive', 'Real_Estate']

Depenedent Variable: unsubscribed 
Campaign: No_Opener 
Industry: Automotive


In [ ]:
# pick a variable column to the list above(for now the target variable should be categorical in nature)

selected_dependent_target_variable =  target_variables[1] # label
selected_campaign =  campaign_types[2] # no_opener
selected_industry = industry_types[3] # automotive
print("Depenedent Variable:",selected_dependent_target_variable, "\nCampaign:", selected_campaign, "\nIndustry:", selected_industry)

In [22]:
# filtering for campaign and industry
data = data[(data.campain_type == selected_campaign) & (data.industry == selected_industry)]

In [23]:
# dropping all other dependent variables not being used in analysis as we don't have this data for the test data

# drop 'text' and 'title' for now as not necessarily performing NLP on the data at this type 
# NLP and Computer Vision change scope and scale of the project

X = data.drop(columns=target_variables)


X = X.drop(columns=['text','title'])

# Dropping length to just make prediction based on bin, not specific value, 
# otherwise I would need to separate models, with and without the specific value

# might have to do this for video_length/title_length if going to switch out the bin values 
X = X.drop(columns=['length'])

y = data[selected_dependent_target_variable]

In [24]:
X_temp = X.copy()
X = pd.get_dummies(X)

In [25]:
# creating train test split:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [26]:

rf = RandomForestClassifier()

rf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
data.length_binned.unique()

array(['med', 'small', 'big', 'biggest'], dtype=object)

In [27]:
single_input = X_temp.iloc[0]
single_input = pd.DataFrame(single_input).T

# Change Input Value Here!
shortest = single_input.copy()
bigger = single_input.copy()
biggest = single_input.copy()

shortest.length_binned = 'small'
bigger.length_binned = 'big'
biggest.length_binned = 'biggest'

# single_input.video_length = 'small' # etc
single_input = pd.get_dummies(single_input, columns=['length_binned', 'title_length_binned'])
shortest = pd.get_dummies(shortest, columns=['length_binned', 'title_length_binned'])
bigger = pd.get_dummies(bigger, columns=['length_binned', 'title_length_binned'])
biggest = pd.get_dummies(biggest, columns=['length_binned', 'title_length_binned'])

single_input = single_input.reindex(columns = X.columns, fill_value=0)
shortest = shortest.reindex(columns = X.columns, fill_value=0)
bigger = bigger.reindex(columns = X.columns, fill_value=0)
biggest = biggest.reindex(columns = X.columns, fill_value=0)

In [29]:
print("Depenedent Variable:",selected_dependent_target_variable, "\nCampaign:", selected_campaign, "\nIndustry:", selected_industry)

print(f"{selected_dependent_target_variable} probability with current length:",rf.predict_proba(single_input)[0][0])
print(f"{selected_dependent_target_variable} probability with shorter length:",rf.predict_proba(shortest)[0][0])
print(f"{selected_dependent_target_variable} probability with longer length:",rf.predict_proba(bigger)[0][0])
print(f"{selected_dependent_target_variable} probability with longest length:",rf.predict_proba(biggest)[0][0])



Depenedent Variable: unsubscribed 
Campaign: No_Opener 
Industry: Automotive
unsubscribed probability with current length: 0.26
unsubscribed probability with shorter length: 0.35
unsubscribed probability with longer length: 0.43
unsubscribed probability with longest length: 0.35
